# 폴백(fallback)

LLM 애플리케이션에는 LLM API 문제, 모델 출력 품질 저하, 다른 통합 관련 이슈 등 다양한 오류/실패가 존재합니다. 이러한 문제를 우아하게 처리하고 격리하는데 `fallback` 기능을 활용할 수 있습니다.

중요한 점은 fallback 을 LLM 수준뿐만 아니라 전체 실행 가능한 수준에 적용할 수 있다는 것입니다.


## LLM API Error 에 대처 방법

LLM API 오류 처리는 `fallback` 을 사용하는 가장 일반적인 사례 중 하나입니다.

LLM API 에 대한 요청은 다양한 이유로 실패할 수 있습니다. API가 다운되었거나, 속도 제한에 도달했거나, 그 외 여러 가지 문제가 발생할 수 있습니다. 따라서 `fallback` 을 사용하면 이러한 유형의 문제로부터 보호하는 데 도움이 될 수 있습니다.

**중요**: 기본적으로 많은 LLM 래퍼(wrapper)는 오류를 포착하고 재시도합니다. `fallback` 을 사용할 때는 이러한 기본 동작을 해제하는 것이 좋습니다. 그렇지 않으면 첫 번째 래퍼가 계속 재시도하고 실패하지 않을 것입니다.


In [ ]:
%pip install -qU langchain langchain-openai

먼저, OpenAI에서 `RateLimitError` 가 발생하는 경우에 대해 모의 테스트를 해보겠습니다. `RateLimitError` 는 OpenAI API의 **API 호출 비용 제한을 초과했을 때 발생하는 오류** 입니다.

이 오류가 발생하면 일정 시간 동안 API 요청이 제한되므로, 애플리케이션에서는 이에 대한 적절한 처리가 필요합니다. 모의 테스트를 통해 `RateLimitError` 발생 시 애플리케이션이 어떻게 동작하는지 확인하고, 오류 처리 로직을 점검할 수 있습니다.

이를 통해 실제 운영 환경에서 발생할 수 있는 문제를 사전에 방지하고, 안정적인 서비스를 제공할 수 있습니다.


In [21]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

In [22]:
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")  # GET 요청을 생성합니다.
response = httpx.Response(
    200, request=request
)  # 200 상태 코드와 함께 응답을 생성합니다.
# "rate limit" 메시지와 응답 및 빈 본문을 포함하는 RateLimitError를 생성합니다.
error = RateLimitError("rate limit", response=response, body="")

`openai_llm` 변수에 `ChatOpenAI` 객체를 생성하고, `max_retries` 매개변수를 0으로 설정하여 **API 호출비용 제한 등으로 인한 재시도를 방지** 합니다.

`with_fallbacks` 메서드를 사용하여 `anthropic_llm`을 `fallback` LLM으로 설정하고, 이를 `llm` 변수에 할당합니다.


In [23]:
# OpenAI의 ChatOpenAI 모델을 사용하여 openai_llm 객체를 생성합니다.
# max_retries를 0으로 설정하여 속도 제한 등으로 인한 재시도를 방지합니다.
openai_llm = ChatOpenAI(max_retries=0)

# Anthropic의 ChatAnthropic 모델을 사용하여 anthropic_llm 객체를 생성합니다.
anthropic_llm = ChatAnthropic(model="claude-3-opus-20240229")

# openai_llm을 기본으로 사용하고, 실패 시 anthropic_llm을 대체로 사용하도록 설정합니다.
llm = openai_llm.with_fallbacks([anthropic_llm])

In [25]:
# OpenAI LLM을 먼저 사용하여 오류가 발생하는 것을 보여줍니다.
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # "닭이 길을 건넌 이유는 무엇일까요?"라는 질문을 OpenAI LLM에 전달합니다.
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        # 오류가 발생하면 오류를 출력합니다.
        print("에러 발생")

에러 발생


OpenAI API의 비용 제한(rate limit)을 시뮬레이션하고, API 호출비용 제한 오류가 발생했을 때의 동작을 테스트하는 예제입니다.

OpenAI 의 GPT 모델을 시도하는데 에러가 발생했고, fallback 모델인 `Anthropic` 의 모델이 대신 추론을 수행했다는 점을 확인할 수 있습니다.

`with_fallbacks()` 로 대체 모델이 설정되어 있고, 대체 모델이 성공적으로 수행했다면, `RateLimitError` 가 발생하지 않습니다.


In [27]:
# OpenAI API 호출 시 에러가 발생하는 경우 Anthropic 으로 대체하는 코드
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # "대한민국의 수도는 어디야?"라는 질문을 언어 모델에 전달하여 응답을 출력합니다.
        print(llm.invoke("대한민국의 수도는 어디야?"))
    except RateLimitError:
        # RateLimitError가 발생하면 "에러 발생"를 출력합니다.
        print("에러 발생")

content='대한민국의 수도는 서울특별시입니다. \n\n서울은 한반도 중앙에 위치하며, 한강을 끼고 있는 대한민국 최대의 도시입니다. 서울의 인구는 약 1000만 명으로 전체 한국 인구의 약 20%가 서울에 거주하고 있습니다. \n\n서울은 조선시대부터 한국의 수도 역할을 해왔으며, 현재는 정치, 경제, 사회, 문화 등 대한민국의 중심지 역할을 하고 있습니다. 대한민국 정부 주요 기관들이 서울에 위치해 있으며, 다양한 기업의 본사도 서울에 많이 자리잡고 있습니다.\n\n또한 고궁, 박물관, 현대적 건축물 등 새로운 것과 전통적인 것이 조화를 이루는 매력적인 도시로서, 많은 관광객이 방문하는 글로벌 도시이기도 합니다.' response_metadata={'id': 'msg_012yS3DPqGNPoAoyVQR2xrWE', 'content': [ContentBlock(text='대한민국의 수도는 서울특별시입니다. \n\n서울은 한반도 중앙에 위치하며, 한강을 끼고 있는 대한민국 최대의 도시입니다. 서울의 인구는 약 1000만 명으로 전체 한국 인구의 약 20%가 서울에 거주하고 있습니다. \n\n서울은 조선시대부터 한국의 수도 역할을 해왔으며, 현재는 정치, 경제, 사회, 문화 등 대한민국의 중심지 역할을 하고 있습니다. 대한민국 정부 주요 기관들이 서울에 위치해 있으며, 다양한 기업의 본사도 서울에 많이 자리잡고 있습니다.\n\n또한 고궁, 박물관, 현대적 건축물 등 새로운 것과 전통적인 것이 조화를 이루는 매력적인 도시로서, 많은 관광객이 방문하는 글로벌 도시이기도 합니다.', type='text')], 'model': 'claude-3-opus-20240229', 'role': 'assistant', 'stop_reason': 'end_turn', 'stop_sequence': None, 'type': 'message', 'usage': Usage(input_tokens=22, output_tokens=339)}


`llm.with_fallbacks()` 설정한 모델도 일반 runnable 모델과 동일하게 동작합니다.

아래의 코드 역시 "오류 발생"은 출려되지 않습니다. fallbacks 모델이 잘 수행했기 때문입니다.


In [31]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "질문에 짧고 간결하게 답변해 주세요.",  # 시스템 역할 설명
        ),
        ("human", "{country} 의 수도는 어디입니까?"),  # 사용자 질문 템플릿
    ]
)
chain = prompt | llm  # 프롬프트와 언어 모델을 연결하여 체인 생성
# chain = prompt | ChatOpenAI() # 이 코드이 주석을 풀고 실행하면 "오류 발생" 문구가 출력됩니다.
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"country": "대한민국"}))  # 체인을 호출하여 결과 출력
    except RateLimitError:  # API 비용 제한 오류 처리
        print("오류 발생")

content='대한민국의 수도는 서울특별시입니다.' response_metadata={'id': 'msg_013FiBouyK7dRti21HMLRvwR', 'content': [ContentBlock(text='대한민국의 수도는 서울특별시입니다.', type='text')], 'model': 'claude-3-opus-20240229', 'role': 'assistant', 'stop_reason': 'end_turn', 'stop_sequence': None, 'type': 'message', 'usage': Usage(input_tokens=46, output_tokens=23)}


## 처리해야할 오류를 구체적으로 명시한 경우

특정 오류를 처리하기 위해 `fallback` 이 호출되는 시점을 더 명확하게 지정할 수도 있습니다. 이를 통해 `fallback` 메커니즘이 동작하는 상황을 보다 세밀하게 제어할 수 있습니다.

예를 들어, 특정 예외 클래스나 오류 코드를 지정함으로써 해당 오류가 발생했을 때만 fallback 로직이 실행되도록 설정할 수 있습니다. 이렇게 하면 **불필요한 `fallback` 호출을 줄이고, 오류 처리의 효율성을 높일 수** 있습니다.

아래의 예제에서는 "오류 발생" 문구가 출력됩니다. 이유는 `exceptions_to_handle` 파라미터에서 `KeyboardInterrupt` 예외가 발생시에만 `fallback` 이 구동되도록 설정했기 때문입니다. 따라서, `KeyboardInterrupt` 를 제외한 모든 예외에서는 `fallback` 이 발생하지 않습니다.


In [33]:
llm = openai_llm.with_fallbacks(
    # 대체 LLM으로 anthropic_llm을 사용하고, 예외 처리할 대상으로 KeyboardInterrupt를 지정합니다.
    [anthropic_llm],
    exceptions_to_handle=(KeyboardInterrupt,),  # 예외 처리 대상을 지정합니다.
)

# 프롬프트와 LLM을 연결하여 체인을 생성합니다.
chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        # 체인을 호출하여 결과를 출력합니다.
        print(chain.invoke({"country": "대한민국"}))
    except RateLimitError:
        # RateLimitError 예외가 발생하면 "오류 발생"를 출력합니다.
        print("오류 발생")

content='대한민국의 수도는 서울특별시입니다.' response_metadata={'id': 'msg_01UbBNaKkSPecHATCKVwyMHQ', 'content': [ContentBlock(text='대한민국의 수도는 서울특별시입니다.', type='text')], 'model': 'claude-3-opus-20240229', 'role': 'assistant', 'stop_reason': 'end_turn', 'stop_sequence': None, 'type': 'message', 'usage': Usage(input_tokens=46, output_tokens=23)}


## fallback 에 여러 모델을 순차적으로 지정

`fallback` 모델에 1가지 모델만 지정할 수 있는 것은 아니고, 여러 개의 모델을 지정 가능합니다. 이렇게 여러개의 모델을 지정했을 때 순차적으로 시도하게 됩니다.


In [37]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 프롬프트 생성
prompt_template = (
    "질문에 짧고 간결하게 답변해 주세요.\n\nQuestion:\n{question}\n\nAnswer:"
)
prompt = PromptTemplate.from_template(prompt_template)

오류를 발생하는 chain 과 정상적인 chain 2가지를 생성합니다.


In [55]:
# 여기서는 쉽게 오류를 발생시킬 수 있는 잘못된 모델 이름을 사용하여 체인을 생성할 것입니다.
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = prompt | chat_model

In [58]:
# fallback 체인을 생성합니다.
fallback_chain1 = prompt | ChatOpenAI(model="gpt-3.6-turbo") # 오류
fallback_chain2 = prompt | ChatOpenAI(model="gpt-3.5-turbo") # 정상
fallback_chain3 = prompt | ChatOpenAI(model="gpt-4-turbo-preview") # 정상

In [59]:
# 두 개의 체인을 결합하여 최종 체인을 생성합니다.
chain = bad_chain.with_fallbacks(
    [fallback_chain1, fallback_chain2, fallback_chain3])
# 생성된 체인을 호출하여 입력값을 전달합니다.
chain.invoke({"question": "대한민국의 수도는 어디야?"})

AIMessage(content='서울입니다.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 46, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})